# GET Airports for cities in europe

In [19]:
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import time
import yaml

In [20]:
# Environment-variables in .env-Datei
load_dotenv()

# API-Key
api_key = os.getenv("AVIATION_KEY")

# API URL
base_url = "https://aviation-edge.com/v2/public/autocomplete"

In [21]:
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
    target_cities = config["cities"]

In [22]:
# airports in europe csv
airports_df = pd.read_csv("wikipedia_airports_eu.csv")

In [23]:
filtered_df = airports_df[airports_df["City served"].isin(target_cities)]
city_country_pairs = filtered_df[['City served', 'Country']].drop_duplicates()

In [24]:
# Dictionary to store airport data for each city
city_data = {}

# Loop through each unique city-country pair
for _, row in city_country_pairs.iterrows():
    city = row['City served']
    country = row['Country']
    
    # Define the API request URL
    url = f"{base_url}?key={api_key}&city={city}"
    
    # Make the request to the API
    response = requests.get(url)
    
    if response.status_code == 200:
        response_data = response.json().get('airportsByCities', [])
        
        # Filter results where nameCountry matches the 'Country' from the CSV and codeIcaoAirport is not empty
        filtered_data = [
            item for item in response_data 
            if item.get('nameCountry') == country and item.get('codeIcaoAirport')
        ]
        
        # Store only if there are matching results
        if filtered_data:
            city_data[city] = filtered_data
            print(f"Data for {city} in {country} retrieved successfully.")
        else:
            print(f"No matching data for {city} in {country}.")
    
    else:
        print(f"Failed to retrieve data for {city}. Status code: {response.status_code}")
    
    # Wait 1 second between API requests to avoid rate limiting
    time.sleep(1)

# Save the final filtered data to JSON
with open("airport_per_city.json", "w") as outfile:
    json.dump(city_data, outfile)

print("Data has been saved to airport_per_city.json.")


Data for London in United Kingdom retrieved successfully.
Data for Paris in France retrieved successfully.
Data for Amsterdam in Netherlands retrieved successfully.
Data for Madrid in Spain retrieved successfully.
Data for Frankfurt in Germany retrieved successfully.
Data for Rome in Italy retrieved successfully.
Data for Moscow in Russia retrieved successfully.
Data for Lisbon in Portugal retrieved successfully.
Data for Dublin in Ireland retrieved successfully.
Data for Vienna in Austria retrieved successfully.
Data for Zurich in Switzerland retrieved successfully.
Data has been saved to airport_per_city.json.


In [25]:
# Save the final filtered data to JSON
with open("airport_per_city.json", "w") as outfile:
    json.dump(city_data, outfile)

In [26]:
with open("airport_per_city.json", "w") as outfile:
    json.dump(city_data, outfile)

In [27]:
# Get all iatacodes for the airports
with open("airport_per_city.json", "r") as infile:
    city_data = json.load(infile)

# Extract all unique codeIataAirport values
iata_codes = {
    "iatacodes": [
        airport["codeIataAirport"]
        for airports in city_data.values()
        for airport in airports
        if "codeIataAirport" in airport and airport["codeIataAirport"]
    ]
}

# IATA codes to iatacodes.json
with open("iatacodes.json", "w") as outfile:
    json.dump(iata_codes, outfile)

print("IATA codes have been saved to iatacodes.json.")

IATA codes have been saved to iatacodes.json.
